In [ ]:
import numpy as np
import pandas as pd
import glob
import os

import torch
import torch.utils.data as data_utils
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from random import shuffle
import json
from sklearn.metrics import mean_squared_error

In [ ]:
# provided by Naay
current_over = None
ball_count = 0

# Function to generate dynamic over numbers
def generate_over(row):
    global current_over, ball_count
    if row['Over'] != current_over:
        current_over = row['Over']
        ball_count = 1
    else:
        ball_count += 1
    return f"{current_over}.{ball_count}"


In [ ]:
def generate_ball_no(row):
    over = int(row['Over'].split('.')[0])
    ball = int(row['Over'].split('.')[1])

    return over*6 + ball

In [ ]:
!unzip /content/Analysis\ T20\ data.zip

In [ ]:
!unzip /content/t20s_csv.zip

In [ ]:
target_dict = {}
target_file = open('/content/new_label_var_alpha_40_24.csv', 'r')
lines = target_file.readlines()
for line in lines:
    line_ = line.split(',')
    target_dict[line_[0]] = [float(x) for x in line_[2:]]

In [ ]:
# T20 Data
path = r'/content/t20s_csv/'
all_files = glob.glob(path + "/*.csv")
# shuffle(all_files)
innings2 = []
target=[]
true_targets=[]

# print (len(all_files))

for filename in all_files:
    json_path = "/content/t20s_json"
    indiv_fname = filename.split('/')[-1][:-4] + '.json'
    json_fname = json_path + '/' + indiv_fname

    df = pd.read_csv(filename)
    f = open(json_fname)
    data = json.load(f)

    # to handle cases like No Result, Rain Stopped Play, Draw etc
    if "winner" in data["info"]["outcome"].keys():
        winteam = data["info"]["outcome"]["winner"]
    else:
        continue

    # first batting team
    f_in_team = df['Team'].iloc[0]
    innings = []
    for idx, val in enumerate(df.iterrows()):
        if df['Team'].iloc[idx] == f_in_team:
            innings.append(1)
        else:
            innings.append(2)

    df["innings"] = innings

    # Apply the function to generate dynamic over numbers
    df['Over'] = df.apply(generate_over, axis=1)

#     print (df.columns)
#     display (df.tail())

    df1 = df[df['innings']==1]
    df1['Team_Runs'] = df1['Total_Runs'].cumsum()
    target_score = df1['Team_Runs'].iloc[-1] +1

    df2 = df.loc[:, ['Over', 'Total_Runs', 'Batter', 'Non_Striker']].set_index(df["innings"]).drop(index=1)

    df2 = df2.reset_index()
    df2 = df2.drop(["innings"], axis = 1)

    df2['Ball'] = df2.apply(generate_ball_no, axis=1)
    df2['Team_Runs'] = df2['Total_Runs'].cumsum()
    df2['To_Win_Runs'] = target_score - df2['Team_Runs']

#     print (df2.columns)

    if df2.shape[0]:
#         df2 = df2.rename(columns={"Over": 0, "Batter_Runs": 1, "Extras": 2})
        innings2.append(df2)
        target.append(target_dict[indiv_fname])
        if winteam == f_in_team:
            true_targets.append(0)
        else:
            true_targets.append(1)



innings2[0]

In [ ]:
import numpy as np
x = np.array(innings2[0].values, dtype=np.float16)
torch.tensor(x)

In [ ]:
print(type(innings2[0]))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
print(len(target[0]))
print(len(true_targets))

104
2910


In [ ]:
innings2[0]

,Over,Total_Runs,Batter,Non_Striker,Ball,Team_Runs,To_Win_Runs
0,0.1,1,AJ Finch,DA Warner,1,1,118
1,0.2,0,AJ Finch,DA Warner,2,1,118
2,0.3,3,AJ Finch,DA Warner,3,4,115
3,0.4,0,DA Warner,AJ Finch,4,4,115
4,0.5,0,DA Warner,AJ Finch,5,4,115
...,...,...,...,...,...,...,...
90,14.6,2,MC Henriques,TM Head,90,114,5
91,15.1,1,TM Head,MC Henriques,91,115,4
92,15.2,1,TM Head,MC Henriques,92,116,3
93,15.3,2,MC Henriques,TM Head,93,118,1


In [ ]:
! pip install ordered_set

In [ ]:
from ordered_set import OrderedSet

In [ ]:

def get_batting_pos_dict(df):
    match_pos_dict = {}
    pos_count = 1
    ns_list = df['Non_Striker'].unique()

    batting_order_set = OrderedSet()
    for i in range(len(df)):
        batting_order_set.add(df['Batter'][i])
        batting_order_set.add(df['Non_Striker'][i])

    for index in range(len(batting_order_set)):
        match_pos_dict[batting_order_set[index]] = index + 1

    return match_pos_dict

In [ ]:
def create_batting_pos_dict(train_files, root = './'):
    batting_pos_arr_dict = {}

    for file in train_files:
        df = pd.read_csv(os.path.join(root, file))
        match_pos_dict = {}
        pos_count = 1
        for i in range(len(df)):
            b = df['Batter'][i]

            if b not in batting_pos_arr_dict.keys():
                batting_pos_arr_dict[b] = np.zeros(12)

            if i!=0 and df.iloc[i-1,0] != df.iloc[i,0]: # innings change
                pos_count = 1

            if b not in match_pos_dict.keys():
                match_pos_dict[b] = pos_count
                pos_count +=1

        for b, pos in match_pos_dict.items():
            batting_pos_arr_dict[b][pos-1] +=1

    batting_pos_dict = {}

    for b in batting_pos_arr_dict.keys():
        batting_pos_dict[b] = np.argmax(batting_pos_arr_dict[b]) +1

    return batting_pos_dict

In [ ]:
def create_batter_avg_dict(train_files, root):

    batter_runs_dict = {}
    batter_inns_dict = {}

    batter_set = set()

    for file in train_files:
        df = pd.read_csv(os.path.join(root, file))

        batter_list = df['Batter'].unique()
        batter_set = batter_set.union(set(batter_list))

        for b in batter_list:
            df_b = df[df['Batter']==b]
            batter_team = df_b['Team'].iloc[0]
            df_team = df[df['Team']==batter_team]

            is_not_out = (df_team['Batter'].iloc[-1] == b) | (df_team['Non_Striker'].iloc[-1] == b)
            batter_runs = df_b['Batter_Runs'].sum()
            batter_balls = len(df_b)

            if b in batter_runs_dict.keys():
                batter_runs_dict[b] += batter_runs
                if not is_not_out:
                    batter_inns_dict[b] += 1
            else:
                batter_runs_dict[b] = batter_runs
                if not is_not_out:
                    batter_inns_dict[b] = 1
                else:
                    batter_inns_dict[b] = 0

    batters_avg = {}
    for b in batter_set:
        if batter_inns_dict[b] > 0:
            batters_avg[b] = batter_runs_dict[b]/batter_inns_dict[b]
        else:
             batters_avg[b] = batter_runs_dict[b]

    return batters_avg

def create_batter_sr_dict(train_files, root):
    batter_runs_dict = {}
    batter_balls_dict = {}
    batter_set = set()

    for file in train_files:
        df = pd.read_csv(os.path.join(root, file))

        batter_list = df['Batter'].unique()
        batter_set = batter_set.union(set(batter_list))

        for b in batter_list:
            df_b = df[df['Batter']==b]
            batter_runs = df_b['Batter_Runs'].sum()
            batter_balls = len(df_b)

            if b in batter_runs_dict.keys():
                batter_runs_dict[b] += batter_runs
                batter_balls_dict[b] += batter_balls
            else:
                batter_runs_dict[b] = batter_runs
                batter_balls_dict[b] = batter_balls

    batters_sr = {}
    for b in batter_set:
        batters_sr[b] = batter_runs_dict[b]/batter_balls_dict[b]*100

    return batters_sr

def create_batter_stats(train_files, root = './'):

    batter_avg_dict = create_batter_avg_dict(train_files, root)
    batter_sr_dict = create_batter_sr_dict(train_files, root)

    return batter_avg_dict, batter_sr_dict


In [ ]:
def create_group_batter_stats(batters_avg, batters_sr, batting_pos_dict):
    grp_avg = np.zeros(3)
    grp_sr = np.zeros(3)
    grp_count = np.zeros(3)

    batters = batting_pos_dict.keys()

    for b in batters:
        if batting_pos_dict[b] <=6:
            grp_avg[0] += batters_avg[b]
            grp_sr[0] += batters_sr[b]
            grp_count[0] +=1
        elif batting_pos_dict[b] >= 7 and batting_pos_dict[b] <= 9:
            grp_avg[1] += batters_avg[b]
            grp_sr[1] += batters_sr[b]
            grp_count[1] +=1
        else:
            grp_avg[2] += batters_avg[b]
            grp_sr[2] += batters_sr[b]
            grp_count[2] +=1

    grp_avg /= grp_count
    grp_sr /= grp_count

    return grp_avg, grp_sr

In [ ]:
def get_batter_features(batter, batting_pos, batter_stats, batter_group_stats):
    batter_avg_dict, batter_sr_dict = batter_stats
    grp_avg, grp_sr = batter_group_stats

    if batter in batter_avg_dict.keys():
        return (batter_avg_dict[batter], batter_sr_dict[batter])
    else:
        # get group features
        if batting_pos <=6:
            return grp_avg[0], grp_sr[0]
        elif batting_pos >= 7 and batting_pos <= 9:
            return grp_avg[1], grp_sr[1]
        else:
            return grp_avg[2], grp_sr[2]

In [ ]:
all_files = os.listdir('/content/t20s_csv/')
split = int(len(all_files) * 0.8)
train_files = all_files[:split]
test_files = all_files[split:]

In [ ]:
train_batting_pos_dict = create_batting_pos_dict(train_files, '/content/t20s_csv/')

In [ ]:
train_batter_avg_dict, train_batter_sr_dict = create_batter_stats(train_files, '/content/t20s_csv/')


In [ ]:
train_grp_avg, train_grp_sr = create_group_batter_stats(train_batter_avg_dict, train_batter_sr_dict, train_batting_pos_dict)

In [ ]:
# torch tensor processing
features=[]
# create targets first
for i in range(len(innings2)):
#     print (i)
    innings2_p1 = innings2[i].loc[:, ['Over', 'To_Win_Runs']]
    x1 = np.array(innings2_p1.values, dtype=np.float16)

    batting_pos_dict = get_batting_pos_dict(innings2[i])
    batter_stats_all = []

#     print (batting_pos_dict)

    for j in range(len(innings2[i])):
        batter = innings2[i]['Batter'].iloc[j]
        batter_batting_pos = batting_pos_dict[batter]

        ns = innings2[i]['Non_Striker'].iloc[j]
        ns_batting_pos = batting_pos_dict[ns]

        batter_features = get_batter_features(batter, batter_batting_pos,
                                              (train_batter_avg_dict, train_batter_sr_dict),
                                              (train_grp_avg, train_grp_sr))
        nonstriker_features = get_batter_features(ns, ns_batting_pos,
                                                  (train_batter_avg_dict, train_batter_sr_dict),
                                                  (train_grp_avg, train_grp_sr))

        batter_stats_all.append([*batter_features, *nonstriker_features])

    x2 = np.array(batter_stats_all)
#     print (x2.shape)
#     print (x1.shape)
    x = np.concatenate((x1,x2), axis=1)

    features.append(torch.tensor(x))




In [ ]:
features[0].shape

torch.Size([104, 6])

In [ ]:
len(features)

2910

In [ ]:
# convert to fixed length sequence
features = pad_sequence(features,batch_first=True, padding_value=-1)
# targets = torch.tensor(target)

max_len = 0
for i in range(len(target)):
    max_len = max(max_len, len(target[i]))

targets = np.zeros((len(target), max_len), dtype='float')
# Padding by true label
for i in range(len(target)):
    target[i].extend([true_targets[i]] * (max_len - len(target[i])))
    targets[i] = np.array(target[i], dtype='float')

targets = torch.FloatTensor(targets)
print(targets.shape)

X_train = features[:split]
X_test  = features[split:]
y_train = targets[:split]
y_test  = targets[split:]
true_targets = torch.tensor(true_targets)
true_y_train = true_targets[:split]
true_y_test  = true_targets[split:]

print(X_train.size())
print(X_test.size())

torch.Size([2910, 160])
torch.Size([2401, 160, 6])
torch.Size([509, 160, 6])


In [ ]:
y_train.shape, y_test.shape

(torch.Size([2401, 160]), torch.Size([509, 160]))

In [ ]:
# LSTM model
class matchRNN(nn.Module):
    def __init__(self,insize,hsize,outsize):
        super(matchRNN,self).__init__()

        self.insize=insize
        self.hsize=hsize
        self.outsize = outsize

        # lstm cell
        self.lstm_cell = nn.LSTMCell(input_size=insize, hidden_size=hsize)
        self.fc_out = nn.Linear(in_features=hsize, out_features=outsize)
#         self.dropout = nn.Dropout(p=0.2, inplace=False)
        self.softmax = nn.Softmax(dim=1)

    def forward(self,feat):
#         feat = torch.tensor(feat[np.newaxis,:],dtype=torch.float32)
        batch_size = feat.size(0)
        # init the hidden and cell states to zeros
        hidden_state = torch.zeros((batch_size, self.hsize))
        cell_state = torch.zeros((batch_size, self.hsize))
        outputs = torch.empty((batch_size, feat.size(1), self.outsize))

        for t in range(feat.size(1)):

            # for the first time step (if input is different)
            if t == 0:
                hidden_state, cell_state = self.lstm_cell(feat[:,t,:].view(batch_size,-1).float(), (hidden_state, cell_state))

            # for the 2nd+ time step
            else:
                hidden_state, cell_state = self.lstm_cell(feat[:,t,:].view(batch_size,-1).float(), (hidden_state, cell_state))

#             dropouts = self.dropout(hidden_state)
            out = self.fc_out(hidden_state)
#             out = self.softmax(out)
            outputs[:,t,:] = out

        return outputs

In [ ]:
def evalfunc(model,j):
    model.eval()
    # train
    X = X_train
    y = y_train
    mean_rmse_train = 0
    num_batches = X.size(0) // batch_size
    softmax = nn.Softmax(dim=1)

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx+1) * batch_size
        op = model(X[start_idx:end_idx])
        req_op = op[:,j-1]
        # maxval,maxidx = torch.max(req_op,1)
        # Normalize the scores and taking the win prob
        req_op = softmax(req_op)
        req_op = req_op[:, 1]
        # RMSE
        mean_rmse_train += mean_squared_error(y[start_idx:end_idx, j-1], req_op.detach().numpy(), squared=False)

    # Mean RMSE for all the batches
    mean_rmse_train = mean_rmse_train / num_batches

    # test
    X = X_test
    y = y_test
    mean_rmse_test = 0
    num_batches = X.size(0) // batch_size

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx+1) * batch_size
        op = model(X[start_idx:end_idx])
        req_op = op[:,j-1]
        # maxval,maxidx = torch.max(req_op,1)
        # Normalize the scores and taking the win prob
        req_op = softmax(req_op)
        req_op = req_op[:, 1]
        # RMSE
        mean_rmse_test += mean_squared_error(y[start_idx:end_idx, j-1], req_op.detach().numpy(), squared=False)

    # Mean RMSE for all the batches
    mean_rmse_test = mean_rmse_test / num_batches
    print('j = {}, Mean Train RMSE = {}, Mean Test RMSE = {}'.format(j,mean_rmse_train,mean_rmse_test))

In [ ]:
def evalfunc_2(model,j):
    model.eval()
    # train
    X = X_train
    y = true_y_train
    corr=0
    num_batches = X.size(0) // batch_size

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx+1) * batch_size
        op = model(X[start_idx:end_idx])
        req_op = op[:,j-1]
        maxval,maxidx = torch.max(req_op,1)
        corr+= np.sum((maxidx==y[start_idx:end_idx]).numpy())
    total=num_batches*batch_size
    train_acc = corr / total

    # test
    X = X_test
    y = true_y_test
    corr=0
    num_batches = X.size(0) // batch_size

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx+1) * batch_size
        op = model(X[start_idx:end_idx])
        req_op = op[:,j-1]
        maxval,maxidx = torch.max(req_op,1)
        corr+= np.sum((maxidx==y[start_idx:end_idx]).numpy())
    total=num_batches*batch_size
    test_acc = corr / total
    print('j = {}, Train Acc = {}, Test Acc = {}'.format(j,train_acc,test_acc))

In [ ]:
new_targets = np.zeros((targets.size(0), targets.size(1),2),dtype='float')
for dim_1 in range(targets.size(0)):
    for dim_2 in range(targets.size(1)):
        new_targets[dim_1][dim_2][1] = targets[dim_1, dim_2]
        if(new_targets[dim_1][dim_2][1] < -0.5):
            new_targets[dim_1][dim_2][0] = new_targets[dim_1][dim_2][1]
        else:
            new_targets[dim_1][dim_2][0] = 1.0 - new_targets[dim_1][dim_2][1]

new_targets = torch.FloatTensor(new_targets)

In [ ]:
print(new_targets[5][90])

tensor([0.9801, 0.0199])


In [ ]:
from tqdm import tqdm_notebook

In [ ]:
### training parameters

insize = X_train.size(2)
hsize = 64
outsize = 2    #binary classification
model = matchRNN(insize,hsize,outsize)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
batch_size = 16
num_batches = int(X_train.size(0) / batch_size)

# train iterations
for epoch in range(121):  # optimum 100-150 epochs
    epoch_loss = 0
    model.train()
    for batch_idx in tqdm_notebook(range(num_batches)):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx+1) * batch_size
        model.zero_grad()
        outputs = model(X_train[start_idx:end_idx])
        loss = loss_function(outputs[:,30:160,:].contiguous().view(-1,outsize),  new_targets[start_idx:end_idx,30:160,:].contiguous().view(-1,outsize))
        loss.backward()
        epoch_loss += loss.data.item()
        optimizer.step()
    print('Epoch = {}, loss = {}'.format(epoch,epoch_loss))
    if epoch % 10 == 0:
        for j in [60, 90, 120, 160]:
            evalfunc(model,j)
            evalfunc_2(model,j)

torch.save(model.state_dict(), '/content/cric_prediction_all_output_batch.pth')


<ipython-input-38-a099b82ec50f>:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_idx in tqdm_notebook(range(num_batches)):


  0%|          | 0/150 [00:00<?, ?it/s]

Epoch = 0, loss = 76.84320618212223
j = 60, Mean Train RMSE = 0.27438253492116926, Mean Test RMSE = 0.30297641792605
j = 60, Train Acc = 0.8125, Test Acc = 0.7540322580645161
j = 90, Mean Train RMSE = 0.3037266312042872, Mean Test RMSE = 0.3294758623646152
j = 90, Train Acc = 0.84875, Test Acc = 0.8245967741935484
j = 120, Mean Train RMSE = 0.3080435341099898, Mean Test RMSE = 0.330231174826622
j = 120, Train Acc = 0.8491666666666666, Test Acc = 0.8387096774193549
j = 160, Mean Train RMSE = 0.24307941868901253, Mean Test RMSE = 0.297474215588262
j = 160, Train Acc = 0.9216666666666666, Test Acc = 0.8810483870967742


<ipython-input-38-a099b82ec50f>:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_idx in tqdm_notebook(range(num_batches)):


  0%|          | 0/150 [00:00<?, ?it/s]

...

Epoch = 118, loss = 33.225334249436855


  0%|          | 0/150 [00:00<?, ?it/s]

Epoch = 119, loss = 32.667517744004726


  0%|          | 0/150 [00:00<?, ?it/s]

Epoch = 120, loss = 32.83460608124733
j = 60, Mean Train RMSE = 0.20493945275743802, Mean Test RMSE = 0.2405877084501328
j = 60, Train Acc = 0.8216666666666667, Test Acc = 0.7943548387096774
j = 90, Mean Train RMSE = 0.22553822775681814, Mean Test RMSE = 0.26841990169017543
j = 90, Train Acc = 0.9070833333333334, Test Acc = 0.8608870967741935
j = 120, Mean Train RMSE = 0.1518151448232432, Mean Test RMSE = 0.1993822281308953
j = 120, Train Acc = 0.96, Test Acc = 0.9274193548387096
j = 160, Mean Train RMSE = 0.032233824948004135, Mean Test RMSE = 0.07951182139579267
j = 160, Train Acc = 0.9941666666666666, Test Acc = 0.9798387096774194
